# Parallel BiasAsker
This notebook shows how to run BiasAsker concurrently

In [1]:
from BiasAsker import *
from apis import *
from multiprocessing.dummy import Process

Create and initialize a BiasAsker object and print its statistics
- "pair ask index": maximum index of absolute bias query that haven't been asked / total number of absolute bias queries
- "pair eval index": maximum index of absolute bias query-answer pair that haven't been evaluated / total number of absolute bias query-answer pairs
- "single ask index": maximum index of relative bias query that haven't been asked / total number of relative bias queries
- "single eval index": maximum index of relative bias query-answer pair that haven't been evaluated / total number of relative bias query-answer pairs
- "id": the id of current slice (start from 0), total number of slices

In [3]:
asker = BiasAsker("en") # for English chatbots, BiasAsker("ch") for Chinese
asker.initialize_from_file("./data/groups_for_auto.csv", "./data/sample_bias_data_for_auto.csv")
asker.get_status()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb3 in position 60: invalid start byte

Partition the BiasAsker into a list of 5 BiasAsker slices (which is the same as a BiasAsker but with an id to identify it for future merging), this will devide the queries evenly among the BiasAsker silices

In [ ]:
asker_list = BiasAsker.partition(5)
for asker_slice in asker_list:
    print(asker_slice.get_status())

Create multiple process to query a chatbot concurrently

In [ ]:
def ask(asker_slice, bot, bot_name):
    print("begin asking", asker_slice.get_status())
    asker_slice.asking_pair_questions(bot, f"./save/{bot_name}")
    asker_slice.asking_single_questions(bot, f"./save/{bot_name}")

bot = BlenderBot()
proc_list = []
for asker_slice in asker_list:
    proc = Process(target=ask, args=(asker_slice, bot, "blender"))
    proc.start()
    proc_list.append(proc)

for proc in proc_list:
    proc.join()


Print the statistics of asker slices

In [ ]:
for asker_slice in asker_list:
    print(asker_slice.get_status())